In [9]:
import os
import time
import random
import itertools
import subprocess
import distutils.util

import numpy as np
import imageio
import gymnasium as gym
import robosuite as suite
from robosuite.wrappers import GymWrapper


In [ ]:
# For Ubuntu
# Configure MuJoCo to use the EGL rendering backend (requires GPU)
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl

# For mac
os.environ["MUJOCO_GL"] = "cgl"   # macOS

# Check if installation was succesful.
try:
  print('Checking that the installation succeeded:')
  import mujoco
  mujoco.MjModel.from_xml_string('<mujoco/>')
except Exception as e:
  raise e from RuntimeError(
      'Something went wrong during installation. Check the shell output above '
      'for more information.\n'
      'If using a hosted Colab runtime, make sure you enable GPU acceleration '
      'by going to the Runtime menu and selecting "Choose runtime type".')

print('Installation successful.')



# Graphics and plotting.
print('Installing mediapy:')
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy
import mediapy as media
import matplotlib.pyplot as plt

# More legible printing from numpy.
np.set_printoptions(precision=3, suppress=True, linewidth=100)

from IPython.display import clear_output
clear_output()

# For macOS

In [ ]:

# Create environment with Franka rob1t
env = suite.make(
    env_name="Lift",
    robots="Panda",
    has_renderer=False,
    has_offscreen_renderer=True,
    use_camera_obs=False
)

# Reset the environment
obs = env.reset()

# Get cube position from observation
cube_pos = obs['cube_pos']
print("Cube position:", cube_pos)

# Define target positions for lifting the cube:
# Stage 1: Move above the cube
above_cube_pos = cube_pos.copy()
above_cube_pos[2] += 0.1  # Position 10cm above the cube

# Stage 2: Move down to grasp the cube
grasp_pos = cube_pos.copy()
grasp_pos[2] -= 0.01  # Move slightly down to grasp

# Stage 3: Lift the cube
lift_pos = cube_pos.copy()
lift_pos[2] += 0.5  # Lift 20cm above the table
#lift_pos[0] += 0.5  # Lift 20cm above the table

print(f"Above cube target: {above_cube_pos}")
print(f"Grasp position: {grasp_pos}")
print(f"Lift position: {lift_pos}")

# Control gains
kp = 5.0  # Position gain
gripper_close_steps = 50  # Steps over which to close gripper gradually
kd = 1.0  # Velocity gain

# Run the simulation
done = False
step_count = 0
max_steps = 500
current_stage = 1
grasped = False

frames = []
while step_count < max_steps:
    current_eef = obs['robot0_eef_pos']
    current_cube_pos = obs['cube_pos']

    # Determine target
    if current_stage == 1:
        target = above_cube_pos
        if np.linalg.norm(current_eef - target) < 0.02:
            current_stage = 2
            print("Above cube reached, moving to grasp")
    elif current_stage == 2:
        target = grasp_pos
        if np.linalg.norm(current_eef - target) < 0.01:
            current_stage = 3
            print("ready to grasp")
    elif current_stage == 3:
        target = grasp_pos
        if action[-1] > 0.5:
            grasped = True
        if np.linalg.norm(current_eef - target) < 0.01 and grasped:
            current_stage = 4 
            print("Cube grasped, lifting")
    else:
        target = lift_pos
        if np.linalg.norm(current_cube_pos - target) < 0.01:
            print("Cube lifted successfully!, current_cube_pos:", current_cube_pos, ", target:", target, ", current_stage:", current_stage)
            break

    # Compute simple proportional action in end-effector space
    action = np.zeros(env.action_dim)
    if current_stage <= 2:
        action[:3] = kp * (target - current_eef)  # x,y,z
        action[3:6] = 0.0  # no rotation change
    else:
        action[:3] = kp * (target - current_cube_pos)  # x,y,z
        action[3:6] = 0.0  # no rotation change
    # Gripper control
    if current_stage <= 2:
        action[-1] = -1.0  # open
    else:
        # Gradually close over first gripper_close_steps
        action[-1] = min(1.0, (step_count % gripper_close_steps) / gripper_close_steps)

    # Step environment
    obs, reward, done, info = env.step(action)
    # Render offscreen frame
    frame = env.sim.render(camera_name="frontview", width=640, height=480)
    frame = np.flip(frame, axis=0)
    frames.append(frame)

    time.sleep(0.01)
    step_count += 1

print("cube_pos:", current_cube_pos, "current_eef:", current_eef, ", target:", target, ", current_stage:", current_stage)
env.close()
print("Simulation finished")
imageio.mimwrite("lift_cube.mp4", frames, fps=30)
print("Video saved!")

[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)


Cube position: [-0.024  0.019  0.83 ]
Above cube target: [-0.024  0.019  0.93 ]
Grasp position: [-0.024  0.019  0.82 ]
Lift position: [-0.024  0.019  1.33 ]
Above cube reached, moving to grasp
ready to grasp
Cube grasped, lifting
Cube lifted successfully!, current_cube_pos: [-0.025  0.02   1.321] , target: [-0.024  0.019  1.33 ] , current_stage: 4
cube_pos: [-0.025  0.02   1.321] current_eef: [-0.029  0.02   1.33 ] , target: [-0.024  0.019  1.33 ] , current_stage: 4
Simulation finished
Video saved!


#### Robosuite envs aren’t directly Gym-compatible, use its GymWrapper and randomize parameters.

In [16]:
class RandomizedLiftEnv(gym.Wrapper):
    """
    Robosuite Lift task with domain randomization:
    - Random cube mass
    - Random cube friction
    """
    def __init__(self):
        env = suite.make(
            env_name="Lift",
            robots="Panda",
            has_renderer=False,
            has_offscreen_renderer=False,
            use_camera_obs=False,
            horizon=200,
            control_freq=20,
            reward_shaping=True,
        )
        super().__init__(GymWrapper(env))

    def reset(self, **kwargs):
        # Sample random parameters
        mass = np.random.uniform(0.05, 0.3)      # 50g – 300g
        friction = np.random.uniform(0.2, 1.0)   # slippery – sticky

        # Access robosuite env from inside wrapper
        env = self.env.env  

        # ✅ Body for mass
        cube_body_id = env.sim.model.body_name2id("cube_main")
        env.sim.model.body_mass[cube_body_id] = mass

        # ✅ Geom for friction (use cube_g0, not cube_main)
        cube_geom_id = env.sim.model.geom_name2id("cube_g0")
        env.sim.model.geom_friction[cube_geom_id] = [friction, 0.005, 0.0001]

        return super().reset(**kwargs)


In [ ]:
from stable_baselines3 import PPO

env = RandomizedLiftEnv()

model = PPO("MlpPolicy", env, verbose=1, tensorboard_log="./ppo_franka_lift/")
model.learn(total_timesteps=200_000)
model.save("ppo_franka_lift")


[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to ./ppo_franka_lift/PPO_1


[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 200      |
|    ep_rew_mean     | 2.54     |
| time/              |          |
|    fps             | 299      |
|    iterations      | 1        |
|    time_elapsed    | 6        |
|    total_timesteps | 2048     |
---------------------------------


[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default

...

[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 200        |
|    ep_rew_mean          | 32.7       |
| time/                   |            |
|    fps                  | 284        |
|    iterations           | 49         |
|    time_elapsed         | 352        |
|    total_timesteps      | 100352     |
| train/                  |            |
|    approx_kl            | 0.01376545 |
|    clip_fraction        | 0.166      |
|    clip_range           | 0.2        |
|    entropy_loss         | -9.24      |
|    explained_variance   | 0.592      |
|    learning_rate        | 0.0003     |
|    loss                 | 0.994      |
|    n_updates            | 480        |
|    policy_gradient_loss | -0.0168    |
|    std                  | 0.906      |
|    value_loss           | 3.33       |
----------------------------------------


[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 200        |
|    ep_rew_mean          | 32.7       |
| time/                   |            |
|    fps                  | 284        |
|    iterations           | 50         |
|    time_elapsed         | 360        |
|    total_timesteps      | 102400     |
| train/                  |            |
|    approx_kl            | 0.01597356 |
|    clip_fraction        | 0.174      |
|    clip_range           | 0.2        |
|    entropy_loss         | -9.24      |
|    explained_variance   | 0.598      |
|    learning_rate        | 0.0003     |
|    loss                 | 1.56       |
|    n_updates            | 490        |
|    policy_gradient_loss | -0.017     |
|    std                  | 0.906      |
|    value_loss           | 3.92       |
----------------------------------------


[robosuite INFO] Loading controller configuration from: /opt/miniconda3/envs/mujoco-env/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)


In [ ]:
import imageio

env = RandomizedLiftEnv()
model = PPO.load("ppo_franka_lift")

frames = []
obs = env.reset()
done = False
while not done:
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    frame = env.render(mode="rgb_array")
    frames.append(frame)

env.close()
imageio.mimsave("ppo_eval.gif", frames, fps=20)
